In [1]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import ParameterGrid

In [2]:
X_train = pd.read_csv("/workspaces/alfonsoMG_boosting/data/processed/diabetes_X_train.csv")
X_test = pd.read_csv("/workspaces/alfonsoMG_boosting/data/processed/diabetes_X_test.csv")
y_train = pd.read_csv("/workspaces/alfonsoMG_boosting/data/processed/diabetes_y_train.csv")
y_test = pd.read_csv("/workspaces/alfonsoMG_boosting/data/processed/diabetes_y_test.csv")

In [3]:
X_train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,10,129.0,62.0,36.0,125.00,41.2,0.441,38
1,0,125.0,68.0,29.0,125.00,24.7,0.206,21
2,1,109.0,56.0,21.0,127.25,25.2,0.833,23
3,6,137.0,61.0,29.0,125.00,24.2,0.151,55
4,9,165.0,88.0,29.0,125.00,30.4,0.302,49


In [4]:
X_test.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,2,100.0,64.0,23.0,125.0,29.7,0.368,21
1,6,190.0,92.0,29.0,125.0,35.5,0.278,66
2,0,93.0,100.0,39.0,72.0,43.4,1.021,35
3,1,97.0,70.0,40.0,125.0,38.1,0.218,30
4,3,96.0,78.0,39.0,125.0,37.3,0.238,40


In [5]:
model = XGBClassifier(random_state = 24)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=24, ...)

In [6]:
y_pred = model.predict(X_train)
accuracy_score(y_train, y_pred)

1.0

In [7]:
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.7142857142857143

In [8]:
model = GradientBoostingClassifier(n_estimators = 10, random_state = 24)
model.fit(X_train, y_train)

/home/vscode/.local/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(n_estimators=10, random_state=24)

In [9]:
y_pred = model.predict(X_train)
accuracy_score(y_train, y_pred)

0.7947882736156352

In [10]:
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.7727272727272727

In [11]:
hyperparameters_x = {
    "n_estimators": [50, 100, 150, 200],
    "learning_rate": [0.01, 0.1, 0.2, 0.3],
    "objective": ["reg:squarederror", "reg:logistic", "binary:logistic"],
    "subsample": [0.8, 0.9, 1.0],
    "max_depth": [3, 5, 7, 10],
    "gamma": [0, 0.1, 0.2],
    "min_child_weight": [1, 3, 5],
    "colsample_by_level": [0.8, 1.0],
    "grow_policy": ["depthwise", "lossguide"],
    "n_jobs": [-1],
    "random_state": [24]
}


In [12]:
accs = []
for combination in ParameterGrid(hyperparameters_x):
    model = XGBClassifier(**combination)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_train)
    accs.append(accuracy_score(y_train, y_pred))

best_acc = max(accs)
best_combination = list(ParameterGrid(hyperparameters_x))[accs.index(best_acc)]
best_model = XGBClassifier(**best_combination)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_train)
accuracy_score(y_train, y_pred)

/home/vscode/.local/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [08:39:26] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "colsample_by_level" } are not used.

  warnings.warn(smsg, UserWarning)
/home/vscode/.local/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [08:39:26] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "colsample_by_level" } are not used.

  warnings.warn(smsg, UserWarning)
/home/vscode/.local/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [08:39:26] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "colsample_by_level" } are not used.

  warnings.warn(smsg, UserWarning)
/home/vscode/.local/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [08:39:26] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "colsample_by_level" } are not used.

  warnings.warn(smsg, UserWarning)
/home/vscode/.local/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [08:39:26] WARNING: /workspac